# Apple store locations

In [1]:
%load_ext lab_black

In [2]:
import requests
import pandas as pd
import geopandas as gpd
from vega_datasets import data
import altair as alt
import json
import datetime as dt

/Users/stiles/.local/share/virtualenvs/notebooks-sxqKeO4C/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [5]:
today = dt.date.today().strftime("%Y-%m-%d")

---

## Grab store list from Apple

In [6]:
params = {
    "locale": "en_US",
}

response = requests.get("https://www.apple.com/rsp-web/store-list", params=params)

In [7]:
data_raw = response.json()["storeListData"]

In [8]:
states = []

for x in data_raw:
    if x["hasStates"] == True:
        for s in x["state"]:
            states.append(dict(state=s, country=x["locale"]))
    else:
        continue

In [9]:
stores = []

for n in states:
    stores.append(
        pd.DataFrame(n["state"]["store"]).assign(country_abbr=n["country"][3:])
    )

In [10]:
src = pd.concat(stores).reset_index(drop=True)

In [11]:
addresses = pd.json_normalize(src["address"])[
    ["address1", "city", "stateName", "stateCode", "postalCode"]
].reset_index(drop=True)

In [12]:
df = pd.concat([src, addresses], axis=1)

In [13]:
df.drop("address", axis=1, inplace=True)

In [14]:
df["url"] = "https://www.apple.com/retail/" + df["slug"]

In [15]:
df.columns = df.columns.str.lower()

In [16]:
countries = {
    "US": "United States",
    "CN": "China",
    "GB": "Great Britain",
    "CA": "Canada",
    "AU": "Australia",
    "FR": "France",
    "IT": "Italy",
    "DE": "Germany",
    "ES": "Spain",
    "JP": "Japan",
    "HK": "Hong Kong",
    "AE": "United Arab Emirates",
    "CH": "Switzerland",
    "TR": "Turkey",
    "NL": "Netherlands",
    "KR": "South Korea",
    "SE": "Sweden",
    "MX": "Mexico",
    "TH": "Thailand",
    "MO": "Macao",
    "TW": "Taiwan",
    "BR": "Brazil",
    "AT": "Austria",
    "BE": "Belgium",
}

In [17]:
df["country"] = df["country_abbr"].map(countries)

In [18]:
df["address"] = (
    df["address1"]
    + ", "
    + df["city"]
    + ", "
    + df["statename"]
    + " "
    + df["postalcode"].fillna("")
)

In [19]:
df.to_csv(f"data/processed/apple_store_locations_{today}.csv", index=False)

In [20]:
df.to_csv(f"data/processed/apple_store_locations.csv", index=False)

---

## Import geocoded

In [21]:
geocoded = pd.read_csv("data/processed/apple_store_locations_geocoded.csv")

In [22]:
geo_df = pd.merge(
    df,
    geocoded[
        [
            "latitude",
            "longitude",
            "formatted_address",
            "input_string",
            "google_place_id",
        ]
    ],
    left_on="address",
    right_on="input_string",
).drop(
    [
        "input_string",
        "google_place_id",
        "country_abbr",
        "statename",
        "address",
        "address1",
        "slug",
        "id",
    ],
    axis=1,
)

In [23]:
geo_df.head()

,name,telephone,city,statecode,postalcode,url,country,latitude,longitude,formatted_address
0,ABQ Uptown,(505) 837-6480,Albuquerque,NM,87110,https://www.apple.com/retail/abquptown,United States,35.102720,-106.566947,"2240 Q St NE, Albuquerque, NM 87110, USA"
1,New Haven,(203) 498-8950,New Haven,CT,06511,https://www.apple.com/retail/newhaven,United States,41.311904,-72.930969,"65 Broadway, New Haven, CT 06511, USA"
2,Greenwich Avenue,(203) 302-6691,Greenwich,CT,06830,https://www.apple.com/retail/greenwichavenue,United States,41.023365,-73.625449,"356 Greenwich Ave, Greenwich, CT 06830, USA"
3,Trumbull,(203) 666-3928,Trumbull,CT,06611,https://www.apple.com/retail/trumbull,United States,41.229341,-73.226774,"5065 Main St, Trumbull, CT 06611, USA"
4,Danbury Fair Mall,(203) 546-6340,Danbury,CT,06810,https://www.apple.com/retail/danburyfairmall,United States,41.376066,-73.484848,"Backus Ave, Danbury, CT 06810, USA"


In [24]:
gdf = gpd.GeoDataFrame(
    geo_df, geometry=gpd.points_from_xy(geo_df.latitude, geo_df.longitude)
)

In [33]:
states = alt.topo_feature(data.us_10m.url, feature="states")
countries = alt.topo_feature(data.world_110m.url, feature="countries")

In [35]:
country = alt.Chart(countries).mark_geoshape(fill="#E9E9E9", stroke="white")

places = (
    alt.Chart(gdf[~gdf["formatted_address"].isnull()])
    .mark_circle(color="#06c")
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.value(8),
        tooltip=["name", "formatted_address", "url"],
    )
    .project("equirectangular")
    .properties(width=900, height=500, title="Apple stores")
)

(country + places).configure_view(strokeWidth=0)

alt.LayerChart(...)

In [36]:
states = alt.topo_feature(data.us_10m.url, feature="states")

places = (
    alt.Chart(gdf[gdf["country"] == "United States"])
    .mark_circle(color="#06c")
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.value(20),
        tooltip=["name", "formatted_address", "url"],
    )
    .project("albersUsa")
    .properties(width=900, height=500, title="Apple stores in the United States")
)

states = (
    alt.Chart(states).mark_geoshape(fill="#E9E9E9", stroke="white").project("albersUsa")
)

(states + places).configure_view(strokeWidth=0)

alt.LayerChart(...)